In [1]:
import tensorflow as tf
import numpy as np

In [2]:
abc = open('./abcnotation/freedomdive.abc', 'r').read()

uniqueText = list(set(abc))
uniqueText.sort()
print(uniqueText)

['\n', ' ', "'", '(', ',', '-', '/', '2', '3', '4', '6', '8', ':', '=', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'K', 'V', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'z', '|']


레이어들은 바보라 숫자만 알아먹어서 글자 -> 숫자 딕셔너리를 만들어줌

In [3]:
textToNum = {}
numToText = {}

for i, data in enumerate(uniqueText):
  textToNum[data] = i
  numToText[i] = data

In [4]:
# 텍스트 전부 숫자로 치환
숫자화 = []

for i in abc:
  숫자화.append(textToNum[i])

print(숫자화)

[13, 20, 4, 6, 7, 13, 15, 4, 6, 7, 13, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 1, 13, 20, 4, 6, 7, 15, 4, 6, 7, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 36, 1, 24, 0, 13, 20, 4, 6, 7, 13, 15, 4, 6, 7, 13, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 1, 13, 20, 4, 6, 7, 15, 4, 6, 7, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 36, 1, 24, 0, 13, 20, 4, 6, 7, 13, 15, 4, 6, 7, 13, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 1, 13, 20, 4, 6, 7, 15, 4, 6, 7, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 36, 1, 24, 0, 13, 20, 4, 6, 7, 13, 15, 4, 6, 7, 13, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 27, 20, 6, 7, 17, 6, 7, 15, 4, 6, 7, 1, 13, 20, 4, 6, 7, 15, 4, 6, 7, 17, 6, 7, 27, 20, 6, 7, 1, 13, 20, 6, 7, 13, 14, 6, 7, 27, 20, 6, 7, 13, 20, 6, 7, 36, 0, 13, 20, 4, 6, 7, 13, 15, 4, 6, 7, 13,

In [5]:
trainX = []
trainY = []

dataLength = len(숫자화)
for i in range (0, dataLength - 25):
  trainX.append(숫자화[i: i+25])
  trainY.append(숫자화[i + 25])

print(np.array(trainX).shape)
print(np.array(trainY).shape)

(18217, 25)
(18217,)


In [6]:
trainX = tf.one_hot(trainX, 31)
trainY = tf.one_hot(trainY, 31)

print(trainX.shape)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-01 16:13:21.514480: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-01 16:13:21.514509: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


(18217, 25, 31)


작곡은 카테고리 예측문제다
- 여러 음중 하나를 뽑아내야 하기 때문 (음 리스트에서 하나 뽑기)

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM( 100, input_shape=(25, 31) ),
  tf.keras.layers.Dense( 31, activation='softmax') 
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               52800     
                                                                 
 dense (Dense)               (None, 31)                3131      
                                                                 
Total params: 55931 (218.48 KB)
Trainable params: 55931 (218.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
saveModel = tf.keras.callbacks.ModelCheckpoint(
  filepath='../../models/checkpoints/RNN_Composing/',
  monitor='val_acc',
  mode='max',
  save_freq='epoch',
)

model.fit(trainX, trainY, batch_size=6, epochs=30, verbose=1, callbacks=[saveModel])
model.save('../../models/RNN_Composing')

Epoch 1/30


2023-06-01 16:13:26.471113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-01 16:13:26.655141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-01 16:13:27.593529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3037/3037 [==============================] - ETA: 0s - loss: 1.5953INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_Composing/assets


INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_Composing/assets


3037/3037 [==============================] - 62s 19ms/step - loss: 1.5953
Epoch 2/30
3036/3037 [============================>.] - ETA: 0s - loss: 1.1871INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_Composing/assets


...

INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_Composing/assets


3037/3037 [==============================] - 60s 20ms/step - loss: 0.8015
Epoch 30/30
3036/3037 [============================>.] - ETA: 0s - loss: 0.7843INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_Composing/assets


INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_Composing/assets


3037/3037 [==============================] - 59s 19ms/step - loss: 0.7843
INFO:tensorflow:Assets written to: ../../models/RNN_Composing/assets


INFO:tensorflow:Assets written to: ../../models/RNN_Composing/assets
